In [1]:
import pypesto
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import petab
import numpy as np
from scipy import integrate
import pickle

In [2]:
datatype = "original"

# Load experimental data
if datatype == "original":
    df=pd.read_csv('data/data.csv', sep='\t')
elif datatype == "switch":
    df=pd.read_csv('data/data_switch.csv', sep='\t')
else:
    df=pd.read_csv('data/data_loss.csv', sep='\t')

add_scaling_to_data = True
scaling_value = 2

if add_scaling_to_data:
    df.Measurement=df.Measurement*scaling_value
    
data = np.asarray(df.Measurement)

tvec = np.asarray(df.Time)
    
N = len(tvec)

In [3]:
#now we interpret kTL_m0 as the scaling factor, therefore we have to take it out of the formula


def analytical_x2(t, t0, xi, delta):
    X = [np.exp(-delta*(t-t0)) * (t>t0),
         (np.exp(-xi*(t-t0)) - np.exp(-delta*(t-t0))) / (delta-xi) * (t>t0)]
    return X[1]

def simulate_model(x, tvec):
    # assign parameters
    t0, xi, delta, _ = x
    # simulate model
    simulation = np.asarray([analytical_x2(t, t0, xi, delta)
                             for t in tvec])
    return simulation

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

In [4]:
def negative_log_marginalised_posterior(x):
    """ Negative log posterior function."""

    shape = x[3]

    # simulate model
    simulation = simulate_model(np.power(10, x), tvec)
    
    
    #we need now to sort the b-vector by size, remove the values for simulation = 0 and then apply
    #this order to the data and simulation as well
    
    #we devide by the accordng values and set -1 where simulation would be 0
    b_vector_us = np.divide(data, simulation, out = -np.ones(N), where=simulation!=0)
    #now these values are at the start and the rest is sorted
    b_vector_l = np.sort(b_vector_us)
    #we safe the order
    sort_order = np.searchsorted(b_vector_l, b_vector_us)
    #and determine how many simulation avlues are 0
    zero_amnt = np.argmax(b_vector_l > -1)
    #they get removed from the b vector
    b_vector = b_vector_l[zero_amnt:]
    
    #N' is the amount of integral breaks
    N_prime = N - zero_amnt
    #we need only N values in the end but add soem spaces for convenience later
    data_sorted = np.ones(N + zero_amnt) - 2
    simulation_sorted = np.ones(N + zero_amnt) - 2
    
    for i in range(N):
        # in case that several values in b_vector have the same value
        j = 0 
        while simulation_sorted[sort_order[i] + j] != -1:
            j += 1
            
        simulation_sorted[sort_order[i] + j] = simulation[i]
        data_sorted[sort_order[i] + j] = data[i]
        
        
    for i in range(zero_amnt):
        data_sorted[N + i] = data_sorted[i]
        simulation_sorted[N + i] = simulation_sorted[i]
        
    data_sorted = data_sorted[zero_amnt:]
    simulation_sorted = simulation_sorted[zero_amnt:]
    
    bounds = np.append(np.append(0, b_vector), np.inf)
    
    d_vector = -np.sum(data_sorted)
    # we can just sum up the whole array because values >= N' are 0 by construction
    q_vector = np.sum(simulation_sorted)
    
    l_llh = 0
    const = lamda*shape
    check = q_vector - const
    
    for i in range(N_prime):
        
        if check != 0:
            aux1 = (shape / check)
            aux2 = np.exp((d_vector + bounds[i + 1] * check) / shape) \
                -  np.exp((d_vector + bounds[i] * check) / shape)
        else:
            aux1 = np.exp(d_vector / shape)
            aux2 = bounds[i + 1] - bounds[i]
        
        l_llh += aux1 * aux2
        d_vector = d_vector + 2*data_sorted[i]
        q_vector = q_vector - 2*simulation_sorted[i]
        check = q_vector - const
        
    l_llh = l_llh - (shape / check) * np.exp((d_vector + bounds[N_prime] * check) / shape)
    
    # return NEGATIVE log posterior (required for pyPESTO)
    return -(np.log(lamda) - N*(np.log(2) + np.log(shape)) + np.log(l_llh))

In [5]:
def posterior(scale, shape, simulation):
    """ Negative log posterior function."""

    diff = -abs(data - scale*simulation) / shape
    factor1 = np.prod(np.exp(diff))
    factor2 = 1 / ((2 * shape)**N)

    llh = factor1 * factor2
    prior = lamda * np.exp(-lamda * scale)

    return llh * prior


In [6]:
def numerical_log_post(shape, simulation):
    numerical_value, _ = integrate.quad(posterior, 0, np.inf, args=(shape, simulation))

    return np.log(numerical_value)

Now that we have the objective function defined, we need to create a pyPESTO problem.

In [7]:
def standard_sampling():
    """Creates a pyPESTO problem."""
    objective = pypesto.Objective(fun=negative_log_marginalised_posterior)
    problem = pypesto.Problem(objective=objective,  # objective function
                              lb=[-2, -5, -5, np.exp(-5)],  # lower bounds
                              ub=[np.log10(df.Time.max()), 5, 5, np.exp(5)],  # upper bounds
                              x_names=['t_0', 'xi', 'delta',
                                       'shape'],  # parameter names
                              x_scales=['log10', 'log10', 'log10',
                                        'lin'])  # parameter scale
    return problem

In [8]:
### Prior dependent paramters

lamda = 0.01

# create the estimation problem
problem = standard_sampling()

# MCMC chain length
n_samples= 1000000

# call the sampler of choice
sampler = sample.AdaptiveMetropolisSampler()

In [9]:
x0 = [np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2]),
      np.array([0.2998, -0.1074, -0.6910, 0.2])]

cov0 = 1e-4

# call the sampler of choice
sampler = sample.AdaptiveParallelTemperingSampler(n_chains=10, internal_sampler=
                                                  sample.AdaptiveMetropolisSampler(options={'cov0': cov0}))

In [10]:
# Define number of runs
runs = 1
save_results = False # for testing just set to False

# Loop over n runs
for n in range(runs):
    # set initial random seed
    np.random.seed(n)
    # perform MCMC sampling
    result = sample.sample(problem, n_samples=n_samples, sampler=sampler, x0=x0)
    # calculate effective sample size
    sample.effective_sample_size(result=result)

    # save the results as a pickle object
    if save_results:
        results = result.sample_result
        with open('Results/Full_parameter/' + str(n) + '.pickle','wb') as result_file:
            pickle.dump(results, result_file, n)

  1%|          | 7402/1000000 [01:17<2:53:55, 95.12it/s] 


LinAlgError: Array must not contain infs or NaNs

There are some built-in visualization functions that one can use.

In [ ]:
print(result.sample_result.effective_sample_size)
print(result.sample_result.effective_sample_size/result.sample_result.time)
# Visualize the parameter trace
ax = visualize.sampling.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))

The scale $s$ is distributed proportional to the density function
\begin{align*}
    \sum_{i = 0}^{N'} \mathbf{1}_{[b_i, b_{i + 1}))}(s) \exp \left\{s \cdot \frac{q_i -\lambda \sigma}{\sigma}\right\}
\end{align*}

The corresponding intgral values are
\begin{align*}
    I_i &= \exp \left\{\frac{d_i}{\sigma} \right\}  c_i \\
    \text{with} \ c_{i = 1, \ldots, N'-1} &=
    \begin{cases}
        \frac{\sigma}{q_i -\lambda \sigma} \left( \exp \left\{b_{i + 1} \cdot \frac{q_i -\lambda \sigma}{\sigma} \right\} - \exp \left\{b_i \cdot \frac{q_i -\lambda \sigma}{\sigma} \right\}\right) &\text{if $q_i \neq \lambda \sigma$} \\
        b_{i + 1} - b_i &\text{if $q_i = \lambda \sigma$}
    \end{cases} \\
    \text{and because $\frac{q_{N'} - \lambda \sigma}{\sigma} < 0$} \quad c_{N'} &= \frac{\sigma}{\lambda \sigma - q_{N'}} \exp \left\{b_{N'} \cdot \frac{q_{N'} -\lambda \sigma}{\sigma} \right\}
\end{align*}

In [ ]:
def get_scale(simulation, shape):    
    
    b_vector_us = np.divide(data, simulation, out = -np.ones(N), where=simulation!=0)
    b_vector_l = np.sort(b_vector_us)
    sort_order = np.searchsorted(b_vector_l, b_vector_us)
    zero_amnt = np.argmax(b_vector_l > -1)
    b_vector = b_vector_l[zero_amnt:]
    bounds = np.append(np.append(0, b_vector), np.inf)
    
    
    N_prime = N - zero_amnt
    data_sorted = np.ones(N + zero_amnt) -2
    simulation_sorted = np.ones(N + zero_amnt) -2
    
    for i in range(N):
        j = 0 # in case that several values in b_vector have the same value
        while simulation_sorted[sort_order[i] + j] != -1:
            j += 1
            
        simulation_sorted[sort_order[i] + j] = simulation[i]
        data_sorted[sort_order[i] + j] = data[i]
        
        
    for i in range(zero_amnt):
        data_sorted[N + i] = data_sorted[i]
        simulation_sorted[N + i] = simulation_sorted[i]
        
    data_sorted = data_sorted[zero_amnt:]
    simulation_sorted = simulation_sorted[zero_amnt:]
    
    
    
    d_vector = -np.sum(data_sorted)
    q_vector = np.sum(simulation_sorted)
    
    probability_mass = np.zeros(N_prime + 1)
    
    const = lamda*shape
    check = q_vector - const
    
    #initial case i = 0
    if check == 0:
        aux1 = np.exp(d_vector / shape)
        aux2 = bounds[1]
        probability_mass[0] = aux1 * aux2
            
    else:
        aux1 = np.exp((bounds[1] * check + d_vector)/ shape)
        aux2 = np.exp(d_vector / shape)
        aux3 = shape / check
        probability_mass[0] = aux3*(aux1 - aux2)
    
    d_vector = d_vector + 2*data_sorted[0]
    q_vector = q_vector - 2*simulation_sorted[0]
    check = q_vector - const
    
    
    
    for i in range(1, N_prime):
        
        if check != 0:
            aux1 = (shape / check)
            aux2 = np.exp((d_vector + bounds[i + 1] * check) / shape) \
                -  np.exp((d_vector + bounds[i] * check) / shape)
        else:
            aux1 = np.exp(d_vector / shape)
            aux2 = bounds[i + 1] - bounds[i]
        
        probability_mass[i] += aux1 * aux2
        d_vector = d_vector + 2*data_sorted[i]
        q_vector = q_vector - 2*simulation_sorted[i]
        check = q_vector - const
        
    normalisation_constant = probability_mass[N_prime-1] - (shape / check) \
            * np.exp((d_vector + bounds[N_prime] * check) / shape)  
    
    probability_mass = probability_mass / normalisation_constant
    probability_mass[N_prime] = 1
    
    s = Generator.uniform(size = 2)
    i = np.argmax(probability_mass > s[0])
    q_vector = np.sum(simulation_sorted[i + 1:]) - np.sum(simulation_sorted[:i + 1])
    factor = (q_vector - lamda * shape) / shape
    
    if factor == 0:
        scale = s[1] * (bounds[i + 1] - bounds[i])
    else:
        lb = bounds[i]

        if i == N_prime:
            scale = np.log(1 - s[1])/ factor + lb
        else:
            ub = bounds[i + 1]
            compensate = factor * ub
            scale = (compensate + np.log(np.exp(factor*lb - compensate) \
                   + s[1] *(np.exp(factor*ub - compensate) - np.exp(factor*lb - compensate)))) / factor
            
    return scale

In [ ]:
Generator = np.random.default_rng()

scale_samples = np.zeros([np.shape(
    result.sample_result.trace_x[0, result.sample_result.burn_in:, 0])[0], 1])

for index, parameter_sample in enumerate(result.sample_result.trace_x[0, result.sample_result.burn_in:, :]):
    shape = parameter_sample[-1]
    simulation = simulate_model(np.power(10, parameter_sample), tvec)
    
    scale_samples[index, :] = get_scale(simulation, shape)
    
if save_results:
    results = [result.sample_result, scale_samples]
    with open('Results/Offset_marginalized/' + str(n) + '.pickle','wb') as result_file:
        pickle.dump(results, result_file, n)

In [ ]:
plt.figure()
ax = sns.kdeplot(scale_samples[:,0], shade=True, color='C0')
plt.xlabel('scale')
plt.ylabel('kde')
plt.show()

In [ ]:
plot_MAP = True

if plot_MAP:
    MAP_index = np.argmax(-result.sample_result.trace_neglogpost[0,result.sample_result.burn_in:])
    MAP = result.sample_result.trace_x[0,result.sample_result.burn_in+MAP_index,:]
    print(MAP)

    tvec_for_simulation = np.linspace(tvec[0],tvec[-1],100)
    
    shape_MAP = MAP[-1]
    
    #changing scale to resampled values
    
    simulation = simulate_model(np.power(10, MAP), tvec)

    scale = get_scale(simulation, shape_MAP)
    print(scale)

    # simulate model
    _simulation = np.asarray(simulate_model(np.power(10, MAP), tvec_for_simulation))
    simulation_for_plotting = np.asarray(scale * _simulation)


    plt.figure()
    plt.plot(tvec,data,'or',label='Corrupted data')
    plt.plot(tvec_for_simulation,simulation_for_plotting,'k',label='MAP simulation')
    plt.xlabel('Time [a.u.]')
    plt.ylabel('Signal [a.u.]')
    #plt.ylim([0,2])
    plt.legend()
    plt.show()